In [1]:
!pip install -qU langchain langchain-openai langchain-cohere rank_bm25 qdrant-client datasets

In [3]:
from datasets import Dataset

In [4]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

Enter your OpenAI API Key:··········


In [5]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

Cohere API Key:··········


In [4]:
!pip install -qU wget

  Preparing metadata (setup.py) ... done


In [5]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv -O john_wick_1.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv -O john_wick_2.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw3.csv -O john_wick_3.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw4.csv -O john_wick_4.csv

--2024-09-27 19:00:07--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19628 (19K) [text/plain]
Saving to: ‘john_wick_1.csv’

john_wick_1.csv     100%[===================>]  19.17K  --.-KB/s    in 0s      

2024-09-27 19:00:07 (99.3 MB/s) - ‘john_wick_1.csv’ saved [19628/19628]

--2024-09-27 19:00:07--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14747 (14K) [text/plain]
Sa

In [6]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"]
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

In [7]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWick"
)

In [8]:
import pandas as pd
testset = pd.read_csv("john_wick_dataset.csv")
test_questions = testset['question'].values.tolist()
test_groundtruths = testset['ground_truth'].values.tolist()

In [14]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

In [9]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

In [10]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model='gpt-4o-mini')

In [11]:
def get_response_dataset(chain, test_questions):
    answers = []
    contexts = []

    for question in test_questions:
        response = chain.invoke({"question" : question})
        answers.append(response["response"].content)
        contexts.append([context.page_content for context in response["context"]])

    response_dataset = Dataset.from_dict({
        "question" : test_questions,
        "answer" : answers,
        "contexts" : contexts,
        "ground_truth" : test_groundtruths
    })

    return response_dataset

In [12]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)

In [16]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

In [29]:
!pip install -q ratelimit

  Preparing metadata (setup.py) ... done


In [30]:
from ratelimit import limits, sleep_and_retry

In [36]:
import time

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
#Install necessary package for rate limiting


# Define a rate limiter to 10 calls per minute
@sleep_and_retry
@limits(calls=10, period=60)
def call_cohere_api(*args, **kwargs):
  """Calls the Cohere API with the given arguments."""
  # Your existing Cohere API call logic goes here
  # For example: self.client.rerank(*args, **kwargs)
  return compression_retriever.get_relevant_documents(*args, **kwargs)

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)
# Update get_response_dataset to use the rate-limited function

contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)


answers = []
contexts = []
for question in test_questions[10:]:
    start_time = time.time()
    print(f"Question: {question}")
    response = contextual_compression_retrieval_chain.invoke({"question" : question})
    # Call the rate-limited function to retrieve documents
    # response["context"] = call_cohere_api(response["question"])
    answers.append(response["response"].content)
    contexts.append([context.page_content for context in response["context"]])
    end_time = time.time()
    print(f"Time taken: {end_time - start_time} seconds")




# contextual_compression_dataset = get_response_dataset(contextual_compression_retrieval_chain, test_questions)

Question: What does the term 'classical orchestra of carnage' imply about the style of John Wick: Chapter 2?
Time taken: 3.240835428237915 seconds
Question: What elements contribute to the unique appeal of John Wick: Chapter 3 - Parabellum?
Time taken: 3.6997203826904297 seconds
Question: What impact has the John Wick franchise had on the standards of action filmmaking in Hollywood?
Time taken: 7.438279628753662 seconds
Question: What event triggers John Wick's quest for revenge in the film?
Time taken: 3.3220598697662354 seconds
Question: What makes John Wick one of the most fun films of the year?
Time taken: 5.1349968910217285 seconds
Question: What does the reference to Baba Yaga suggest about John Wick's character in the third chapter?
Time taken: 3.159041166305542 seconds
Question: What evidence suggests that John Wick is portrayed as a supernatural being in the third chapter of the series?
Time taken: 3.6458890438079834 seconds
Question: What makes the stunts in John Wick 3 consi

In [38]:
for question in test_questions[10:]:
    start_time = time.time()
    print(f"Question: {question}")
    response = contextual_compression_retrieval_chain.invoke({"question" : question})
    # Call the rate-limited function to retrieve documents
    # response["context"] = call_cohere_api(response["question"])
    answers.append(response["response"].content)
    contexts.append([context.page_content for context in response["context"]])
    end_time = time.time()
    print(f"Time taken: {end_time - start_time} seconds")

Question: What power does The Marquis have over John Wick regarding The Table's rules and Wick's past with Santino?
Time taken: 2.5336990356445312 seconds
Question: What role does Wick's car play in his revenge on the mobster's son?
Time taken: 4.339585781097412 seconds
Question: What role does the Continental play in Wick's conflict with Santino?
Time taken: 3.2319188117980957 seconds
Question: What impact do rising headshot fatalities have on character dynamics in crime films?
Time taken: 3.845539093017578 seconds
Question: What adds to the dark style in John Wick's revenge story?
Time taken: 3.475851535797119 seconds
Question: What new choreography and filming techniques did John Wick bring to the industry?
Time taken: 4.564177989959717 seconds
Question: What sets John Wick apart from typical American action films in terms of script and direction?
Time taken: 8.853579044342041 seconds
Question: What makes John Wick different from usual action heroes?
Time taken: 2.7433879375457764 s

In [42]:
contextual_compression_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
    })

In [48]:
contextual_compression_dataset.to_parquet('contextual_compression_dataset.parquet')

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

63985

In [18]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [19]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = Qdrant(
    collection_name="full_documents", embeddings=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

parent_document_retriever.add_documents(parent_docs, ids=None)

<ipython-input-19-3bb3a3dcb6ac>:16: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  parent_document_vectorstore = Qdrant(


In [20]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

In [21]:


ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [22]:
ensemble_reponse_dataset = get_response_dataset(ensemble_retrieval_chain, test_questions)

In [54]:
ensemble_reponse_dataset.to_parquet("ensemble_response_dataset.parquet")

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

272021